In [ ]:
import sisl
import hubbard as hubb
import hubbard.sp2 as sp2
import hubbard.density as density
import hubbard.plot as plot
%matplotlib inline

# Periodic structures (perfect crystals, Bloch’s theorem) 

In this example we will reproduce the results of Ref. [Phys. Rev. B 81, 245402 (2010)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.245402). We will study the effect of on-site Coulomb interactions for electrons in a periodic 1D system by solving the mean-field Hubbard equation using the `hubbard` package.

Consider, for instance, the case of the zigzag graphene nanoribbon 16 C-atoms across width (which we will call 16-ZGNR) with the parameters of set D (see Table I of the [Ref. paper](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.245402)). This is, `t1=2.7`, `t2=0.2`, `t3=0.18` eV for the kinetic part (TB Hamiltonian), and `U=2.0` eV for the interaction part (Hubbard term).

Firstly we build the geometry of the unit cell of the 16-ZGNR with appropiate cell dimensions to have periodicity, e.g., along the $x$-axis direction and no coupling in any other direction.

In [ ]:
# You can use the predefined function of sisl to build the unit cell of a 16-ZGNR:
geom = sisl.geom.zgnr(16)

# This function returns a periodic ZGNR along the x-axis (sisl.Geometry object).
print(geom)

Now you have to build the TB Hamiltonian, just as in the case of example [MFH 1](../MFH_1/run.ipynb). Again, you can make use of the function `sp2` of the `hubbard` package, which will return the `sisl.Hamiltonian` object that we need to build the  `hubb.HubbardHamiltonian`.

In [ ]:
# Build sisl.Hamiltonian object using the sp2 function with the specific hopping parameters
Hsp2 = sp2(...)

In this case, since the system has periodic boundary conditions, the Hamiltonian has to be diagonalized per $\mathbf k$-point and then integrated to find the spin-densities. To do so you just need to pass the argument `nkpt=[nkx, nky, nkz]` when creating the `hubb.HubabrdHamiltonian(...)` object. This argument will set the number of $\mathbf k$-points along each direction in which the Hamiltonian will be sampled in k-space. I.e. if the system is periodic only along the $x$-axis, you should pass something like `nkpt=[nkx, 1, 1]`, where `nkx>1` (the larger this number is, the better the sampling and the slower the convergence process are).

In [ ]:
# Build the HubbardHamiltonian object using the U value from the reference paper
HH = hubb.HubbardHamiltonian(Hsp2, U=U, nkpt=[<nkx>, <nky>, <nkz>], kT=1e-5)

- 1. Before converging, you can plot the band structure of the pure TB Hamiltonian (`Hsp2`). One possible way is to do it with `sisl`. You can take a look at example [TB_01](https://github.com/zerothi/ts-tbt-sisl-tutorial/blob/master/TB_01/run.ipynb). In this case, the system is periodic along the $x$-axis, so you should calculate the bands like:
```
band = sisl.BandStructure(Hsp2, [[0., 0., 0.], [1./2, 0., 0.]], 301, [r'$\Gamma$', 'X'])
```

- 2. Now you can converge using the routine `hubb.HubbardHamiltonian.converge(...)`, just as in example [MFH 1](../MFH_1/run.ipynb). Remember to initialize the spin-densities before convergence! TIP: you can start with polarizing the lower and upper edges of the ZGNR (place up electrons on one border and down electrons on the opposite border).

- 3. Finally, you can visualize relevant physical quantities, such as the spin polarization per unit-cell and the band structure of the self-consistent solution. In exercise [MFH 1](../MFH_1/run.ipynb) there is an example of how to plot the spin polarization using the `hubbard` package. To plot the bandstructure of the converged Hamiltonian you can do the same as in 1., but using the self-consistent solution stored in `HH`: 
```
band = sisl.BandStructure(HH.H, ...)
```

For more information about plotting functionalities in the `hubbard` package you can type `help(plot)`.

Now you can compare the bandstructure of the system for the pure TB Hamiltonian (before convergence) and of the self-consistent solution (after convergence).
You should see a gap opening between the valence and conduction bands with respect to the non-converged solution. Such gap is called the correlation gap, and it appears because of the interaction between electrons.